<a href="https://colab.research.google.com/github/Tekleab15/Context-Aware-Conversational-Search-System/blob/main/notebooks/semantic_search_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implmenting the semantic search and measuring the cosine similarity**

Import all required modules and mounting the drive

In [2]:
# Installing the gradio-UI to show results
# !pip install gradio

In [32]:
import numpy as np
import gradio as gr
import pandas as pd
import tensorflow as tf
import requests
import urllib3
from sentence_transformers import SentenceTransformer,util
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
drive.mount('/content/drive')

print("Run and mounted Successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Run and mounted Successfully!


Loading the embeddings embeded using (SBERT)

In [19]:
# Load the pre-computed embeddings
embeddings_instructions = np.load('/content/drive/My Drive/Conversational_semantic_search/embeddings_instructions.npy')
embeddings_responses = np.load('/content/drive/My Drive/Conversational_semantic_search/embeddings_responses.npy')
model = SentenceTransformer('all-mpnet-base-v2')
print("Model loaded successfully")

data_path = '/content/drive/My Drive/Conversational_semantic_search/Customer_service_preprocessed_.csv'
data_path1 = '/content/drive/My Drive/Conversational_semantic_search/Customer_service.csv'
df = pd.read_csv(data_path)
df1 = pd.read_csv(data_path1)

Model loaded successfully


**Search implementation without using context**

In [28]:
# Response with out using context(memory of prior queries)
def get_response_without_context(user_query):
    global model
    global embeddings_instructions
    global df
    if model is None:
        return "Error: Model not loaded. Try re-running the notebook."
    try:
        query_embedding = model.encode([user_query])
        similarities = cosine_similarity(query_embedding, embeddings_instructions)
        best_match_index = similarities.argmax()
        best_response = df.iloc[best_match_index]['response']
        return best_response
    except Exception as e:
        return f"An error occurred: {str(e)}"


In [29]:
print(get_response_without_context("How can I cancel my order?"))

Assuredly! I understand that you want to cancel the purchase you made. Let me guide you through the process step by step: 

1. Log in to your {{Online Company Portal Info}} using your credentials.
2. Navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section, where you can find a list of your previous purchases.
3. Identify the specific purchase you want to cancel. You may need to search for it by order number or date.
4. Once you've located the purchase, look for an option labeled '{{Online Order Interaction}}' or something similar. Click on it.
5. Follow the instructions provided by the system to complete the cancellation process.

If you encounter any difficulties or have any questions along the way, our dedicated customer support team is here to assist you. You can reach us during {{Customer Support Hours}} at {{Customer Support Phone Number}} or through the Live Chat on our website at {{Website URL}}. We value your satisfaction and will do our best to

**Adding Context (memory)**

In [30]:
# Initializing session memory to store the last 2-3 queries
session_memory = []

def get_response_with_context(user_query):
    global model
    global embeddings_instructions
    global df
    global session_memory

    # Check if the model is loaded
    if model is None:
        return "Error: Model not loaded. Please ensure the model is loaded before querying."
    try:
        # Add the current query to the session memory (limit to the last 4 queries)
        session_memory.append(user_query)
        # Limiting the memory size to last 4 queries
        if len(session_memory) > 4:
            session_memory.pop(0)
        # Combine all previous queries to maintain context
        context = " ".join(session_memory)
        context_embedding = model.encode([context])
        # Calculate cosine similarities between the context and embeddings
        similarities = cosine_similarity(context_embedding, embeddings_instructions)

        # Identify the best matching response based on the highest similarity
        best_match_index = similarities.argmax()
        best_response = df.iloc[best_match_index]['response']
        return best_response
    except Exception as e:
        return f"An error occurred: {str(e)}"


**Adding Intent Recongnition**

In [33]:
candidate_labels = list(df1['intent'].unique())
candidate_embeddings = model.encode(candidate_labels, convert_to_tensor=True)
def predict_intent(user_query):
  try:
    query_embedding = model.encode([user_query], convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, candidate_embeddings)
    best_idx = cos_scores.argmax().item()
    return candidate_labels[best_idx]
  except Exception as e:
    return f"An error occurred: {str(e)}"

**Adding Knowledge-graph**

In [37]:
def get_kg_info(query):
    url = "https://api.dbpedia-spotlight.org/en/annotate"
    headers = {"Accept": "application/json"}
    params = {"text": query, "confidence": 0.5}
    try:
        res = requests.get(url, headers=headers, params=params, verify=False)
        if res.status_code == 200:
            data_kg = res.json()
            if "Resources" in data_kg:
                entities = [resource["@surfaceForm"] for resource in data_kg["Resources"]]
                return ", ".join(entities)
    except Exception as e:
        return f"KG Error: {e}"
    return "No additional knowledge found."

**Adding Recommendation systems**

In [ ]:
response_texts = df['response'].tolist()
response_embeddings = model.encode(response_texts, convert_to_tensor=True)
def recommend_responses(query, top_n=4):
    query_embedding = model.encode([query], convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, response_embeddings)
    top_indices = cos_scores[0].topk(top_n).indices.tolist()
    recommendations = [response_texts[i] for i in top_indices]
    return recommendations

In [38]:
# def chat_with_intent(user_query, response_mode):
def chat_with_intent(user_query, response_mode):
    try:
      intent = predict_intent(user_query)
      kg_info = get_kg_info(user_query)
      recommended = recommend_responses(user_query, top_n=4)
      rec_text = "\nRecommended Responses:\n" + "\n".join(recommended)
      if response_mode == "With Context":
        response = get_response_with_context(user_query)
      else:
          response = get_response_without_context(user_query)
      return f"Intent: {intent}\nKG Info: {kg_info}\nResponse: {response}\n{rec_text}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

iface = gr.Interface(
    fn=chat_with_intent,
    inputs=[
        gr.Textbox(label="Your Query", placeholder="Enter text here..."),
        gr.Radio(["Without Context", "With Context"], label="Response Mode")
    ],
    outputs="text",
    title="Intent Detection with SBERT",
    description="Classifies a user query's intent using SBERT embeddings and cosine similarity."
)
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bdb894e1ff80f2597e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
